# Welcome to the local activation analysis tutorial!

Medusa's local activation module implements several parameters focused on analyzing the signal of each sensor individually. That is, the parameters in this category return a single value for sensor and epoch. The parameters here can be divided in two sub-categories: spectral and non-linear. The **spectral parameters** measure different properties of the signal's power spectral density. On the other hand, the **non-linear parameters** assess the time courses of the signals, evaluating its non-linear properties such as the complexity, irregularity, or variability.

This notebook will cover the main functions of the module through illustrative examples that will allow you to get started in biosignal analysis with Medusa.

In this notebook you will learn:
    - Download an open EEG dataset and explore the files
    - Pre-process the EEG signal with Medusa filters
    - Know the different spectral parameters
    - Know the different non-linear parameters
    - Get some plots with Medusa's plot functions

Do not forget to check the documentation if you do not understand something!

## Imports

Import the modules that will be used in this notebook.

In [38]:
# General imports
import numpy as np
from tabulate import tabulate
import glob

# Medusa imports
from medusa import components
from medusa.meeg import meeg
from medusa import transforms
from medusa.spatial_filtering import LaplacianFilter
from medusa.frequency_filtering import FIRFilter
from medusa.local_activation import spectral_parameteres,nonlinear_parameters

## Download the dataset

As strong supporters of open science, we have released and adapted some
valuable datasets that can be very useful for researchers and practitioners.
These datasets can be downloaded manually from www.medusabci.com/datasets/ or
using a simple API. In this case, we will use the API. Run the following cell
to download the GIB-UVa ERP dataset [1].

Each file is an instance of medusa.data_structures.Recording. This class
contains the information of the performed experiment and the recorded biosignals
In this case, the recordings contain an instance of
medusa.components.ERPSpellerData, which is the default class for this
experiment. Additionally, all recordings contain a medusa.meeg.EEG instance.

In [39]:
#This functionality is not yet implemented, so the code needed to generate a synthetic signal is included.

# dataset_folder = os.getcwd()

# Signal parameters
l_cha = ['F7','F3','FZ','F4','F8','FCz','C3','CZ','C4','CPz','P3','PZ','P4',
         'PO7','POZ','PO8'] # Channel labels
fs = 256 # Sampling frequency
T = 120 # Signal duration
times = np.arange(0,T,1/fs) # Times vector
freqs = [4, 10, 13] # Fundamental signal frequencies
A_freqs = [1.2, 0.8, 0.5] # Frequencies amplitudes
A_noise = 1 # Noise amplitude
sigma = 0.75 # Gaussian noise variance
ps = np.linspace(0,-np.pi/2,len(l_cha)) # Phase differences between channels

# Define signal
signal = np.empty((len(times),len(l_cha)))
for c in range(len(l_cha)):
    signal[:,c] = A_freqs[0] * np.sin(2 * np.pi * freqs[0] * times - ps[c]) + A_freqs[1] * np.sin(2 * np.pi * freqs[1] * times - ps[c]) + A_freqs[2] * np.sin(2 * np.pi * freqs[2] * times - ps[c]) + A_noise * np.random.normal(
            0, sigma, size=times.shape)

# Create a channel set
ch_set = meeg.meeg.EEGChannelSet()
ch_set.set_standard_montage(l_cha=l_cha,montage='10-05')

print("If you have run this cell, you must not run the following cell (Explore de EEG file)")

# Print some info of the extracted features
data_exploration = [
    ['Sampling rate (Hz)', fs],
    ['Channel labels', ch_set.l_cha],
    ['Signal duration (s)', T]]

print('\nData exploration: \n')
print(tabulate(data_exploration))

If you have run this cell, you must run the following cell (Explore de EEG file)

Data exploration: 

-------------------  -----------------------------------------------------------------------------------------------------
Sampling rate (Hz)   256
Channel labels       ['F7', 'F3', 'FZ', 'F4', 'F8', 'FCZ', 'C3', 'CZ', 'C4', 'CPZ', 'P3', 'PZ', 'P4', 'PO7', 'POZ', 'PO8']
Signal duration (s)  120
-------------------  -----------------------------------------------------------------------------------------------------


## Explore the EEG file

Once we have downloaded the EEG recording dataset, we will import the files and extract interesting information such as the channels used, the sampling frequency, or a copy of the signal.

In [40]:
# Load one recording from the dataset
folder = 'data'
file_pattern = '*.rec.bson'
files = glob.glob('%s/%s' % (folder, file_pattern))
recording = components.Recording.load(files[0])

# Get the EEG signal and time-stamps vector from recording
signal = recording.eeg.signal.copy()
times = recording.eeg.times.copy()

# Get the EEG Channel set and sampling rate
ch_set = recording.eeg.channel_set
fs = recording.eeg.fs

# Print some info of the extracted features
data_exploration = [
    ['Sampling rate (Hz)', fs],
    ['Channel labels', ch_set.l_cha],
    ['Signal duration (s)', len(times)/fs]]

print('\nData exploration: \n')
print(tabulate(data_exploration))

IndexError: list index out of range

## Signal Pre-processing

Now it is time to prepare our signal to the incoming analyses. To do so, we have to apply a proper spacial and spectral pre-processing!

Let's suppose that we want to focus our analyses on Fz, C3 and C4 channels. Firstly, we want to emphasize the activity we are interested in. Therefore, we will apply a surface Laplacian spatial filter to eliminate the highly correlated activity of the surrounding channels. Then, a band-pass filter between 0.5 Hz and 30 Hz is then applied to reject the influence of the DC line noise components. We will use a finite impulse response (FIR) filter with a Hamming window.

In [ ]:
# Create an instance of Laplacian filter and fit it
l_cha_to_filter=['FZ','C3','C4']
lp_filter = LaplacianFilter(channel_set=ch_set,mode='auto')
lp_filter.fit_lp(l_cha_to_filter=l_cha_to_filter)
# Mode 'auto' automatically calculates the best small laplacian filter. Let's print
# the channels used to calculate the filter for each studied channel

for i, filter in enumerate(lp_filter.lp_filter):
    print('\n The Laplacian filter for ' + l_cha_to_filter[i] + ' are: ')
    for ch in filter:
        print(f'\n {l_cha[ch]}')


# Apply surface Laplacian filter
signal_lpf = lp_filter.apply_lp(signal.copy())

# Create an instance of FIR filter and fit it
order = 500 # Order of the filter
cutoff = [0.5,30]
btype = 'bandpass' # Type of the filter
window = 'hamming'
filt_method = 'filtfilt' # Filtering method
axis = 0
fir_filter = FIRFilter(order=order,cutoff=cutoff,btype=btype,window=window,
                       filt_method=filt_method,axis=axis)
fir_filter.fit(fs=fs)
# Display the FIR filter
fir_filter.display()

# Apply the FIR filter
signal_f = fir_filter.transform(signal=signal_lpf)

print("The pre-processing stage is finished!")
